In [31]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd

from keras.layers import Dense,Input,LSTM,Bidirectional,Activation,Conv1D
from keras.layers import Dropout,Embedding,GlobalMaxPooling1D, MaxPooling1D, Add, Flatten
from keras.preprocessing import text, sequence
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.models import Model
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [3]:
EMBEDDING_FILE= 'glove.6B.100d.txt'
TRAIN_DATA_FILE= 'train.csv'
TEST_DATA_FILE= 'test.csv'

In [10]:
embed_size = 100 # how big is each word vector
max_features = 100000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 150 # max number of words in a comment to use

In [11]:
train = pd.read_csv(TRAIN_DATA_FILE)
test = pd.read_csv(TEST_DATA_FILE)

list_sentences_train = train["comment_text"].fillna("_na_").values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_test = test["comment_text"].fillna("_na_").values

In [12]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)

In [13]:
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.strip().split()) for o in open(EMBEDDING_FILE, encoding="utf8"))

In [14]:
"""all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
emb_mean,emb_std"""

'all_embs = np.stack(embeddings_index.values())\nemb_mean,emb_std = all_embs.mean(), all_embs.std()\nemb_mean,emb_std'

In [16]:
word_index = tokenizer.word_index
num_words = min(max_features, len(word_index) + 1)

nb_words = min(max_features, len(word_index))
embedding_matrix = np.zeros((num_words, embed_size))#np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [33]:
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
x = SpatialDropout1D(0.2)(x)
x = Bidirectional(GRU(128, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
x = Conv1D(64, kernel_size = 2, padding = "valid", kernel_initializer = "glorot_uniform")(x)
avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
x = concatenate([avg_pool, max_pool])
preds = Dense(6, activation="sigmoid")(x)
model = Model(inp, preds)
model.compile(loss='binary_crossentropy',optimizer=Adam(lr=1e-3),metrics=['accuracy'])
es = callbacks.EarlyStopping(monitor='val_acc', min_delta=0, patience=0, verbose=0, mode='max')

In [34]:
model.fit(X_t, y, batch_size=32, epochs=10, validation_split=0.1, callbacks=[es])

Train on 143613 samples, validate on 15958 samples
Epoch 1/10
143613/143613 [==============================] - 2780s 19ms/step - loss: 0.0549 - acc: 0.9805 - val_loss: 0.0454 - val_acc: 0.9828
Epoch 2/10
143613/143613 [==============================] - 2680s 19ms/step - loss: 0.0413 - acc: 0.9841 - val_loss: 0.0452 - val_acc: 0.9834
Epoch 3/10
143613/143613 [==============================] - 2786s 19ms/step - loss: 0.0357 - acc: 0.9859 - val_loss: 0.0455 - val_acc: 0.9826


In [35]:
y_test = model.predict([X_te], batch_size=1024, verbose=1)
sample_submission = pd.read_csv('sample_submission.csv')
sample_submission[list_classes] = y_test
sample_submission.to_csv('lstm5.csv', index=False)

153164/153164 [==============================] - 521s 3ms/step


In [ ]:
#0.9814